# FPBoost: Fully Parametric Gradient Boosting for Survival Analysis

Source code of the paper "FPBoost: Fully Parametric Gradient Boosting for Survival Analysis" for AAAI 2025.

## Imports

In [1]:
from typing import Optional
from abc import ABC, abstractmethod

import os
import json

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

from sklearn.model_selection import StratifiedKFold
from sklearn.compose import make_column_selector
from sklearndf.pipeline import PipelineDF
from sklearndf.transformation import (
    ColumnTransformerDF,
    OneHotEncoderDF,
    SimpleImputerDF,
    StandardScalerDF,
)
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split

from sksurv.metrics import integrated_brier_score
from sksurv.linear_model import CoxPHSurvivalAnalysis
from sksurv.ensemble import RandomSurvivalForest as RSF, GradientBoostingSurvivalAnalysis

try:
    from auton_survival.models.dsm import DeepSurvivalMachines as DSM
except ImportError:
    os.system("git clone https://github.com/autonlab/auton-survival.git")
    os.system("mv auton-survival/auton_survival .")
    os.system("rm -r auton-survival")
    from auton_survival.models.dsm import DeepSurvivalMachines as DSM

import numba
import torch
from torch import Tensor
from torch.autograd import Variable
import torch.nn.functional as F

import torchtuples as tt
from pycox.models import DeepHitSingle, CoxPH

import ray
from ray import tune
from ray.tune.search.optuna import OptunaSearch
import optuna

from utils.data_loader import load_dataframe


SEED = 42

np.random.seed(SEED)
torch.manual_seed(SEED);

Cloning into 'auton-survival'...


## Data Loading

This section defines the data loading and preprocessing functions alongsiide the cross-validation code.

In [2]:
FOLDS = 10  # Number of folds for cross-validation

In [3]:
def get_preprocess_transformer():
    """Returns the preprocessing sklearn transformer."""
    sel_fac = make_column_selector(pattern="^fac\\_")
    enc_fac = PipelineDF(
        steps=[("ohe", OneHotEncoderDF(sparse_output=False, handle_unknown="ignore"))]
    )
    sel_num = make_column_selector(pattern="^num\\_")
    enc_num = PipelineDF(
        steps=[
            ("impute", SimpleImputerDF(strategy="median")),
            ("scale", StandardScalerDF()),
        ]
    )
    tr = ColumnTransformerDF(transformers=[("ohe", enc_fac, sel_fac), ("s", enc_num, sel_num)])
    return tr


def get_k_fold_splits(df):
    """Returns a generator of k-fold splits."""
    events = df["event"].values.astype(bool)
    times = df["time"].values
    times = times / times.max()

    skf = StratifiedKFold(n_splits=FOLDS, shuffle=True, random_state=SEED)
    splits = list(skf.split(df, events))
    splits = [s[1] for s in splits]

    for i in range(FOLDS):
        val_idx = splits[i]
        test_idx = splits[(i + 1) % FOLDS]
        train_idx = [j for j in range(len(df)) if j not in val_idx and j not in test_idx]

        df_train = df.iloc[train_idx]
        df_val = df.iloc[val_idx]
        df_test = df.iloc[test_idx]

        tr = get_preprocess_transformer()
        X_train = tr.fit_transform(df_train).to_numpy().astype(np.float32)
        X_val = tr.transform(df_val).to_numpy().astype(np.float32)
        X_test = tr.transform(df_test).to_numpy().astype(np.float32)

        e_train = events[train_idx]
        e_val = events[val_idx]
        e_test = events[test_idx]

        t_train = times[train_idx]
        t_val = times[val_idx]
        t_test = times[test_idx]

        max_time = t_train.max()
        min_time = t_train.min()
        X_val = X_val[(min_time < t_val) & (t_val < max_time)]
        e_val = e_val[(min_time < t_val) & (t_val < max_time)]
        t_val = t_val[(min_time < t_val) & (t_val < max_time)]
        X_test = X_test[(min_time < t_test) & (t_test < max_time)]
        e_test = e_test[(min_time < t_test) & (t_test < max_time)]
        t_test = t_test[(min_time < t_test) & (t_test < max_time)]

        sksurv_type = [("event", bool), ("time", float)]
        y_train = np.array([(e, t) for e, t in zip(e_train, t_train)], dtype=sksurv_type)
        y_val = np.array([(e, t) for e, t in zip(e_val, t_val)], dtype=sksurv_type)
        y_test = np.array([(e, t) for e, t in zip(e_test, t_test)], dtype=sksurv_type)

        yield (X_train, y_train), (X_val, y_val), (X_test, y_test)

## Models

This section defines the FPBoost model and the base learners, all implementing the `SurvModel` abstract class.

In [4]:
@numba.njit
def concordance_index_td(
    events: np.ndarray, times: np.ndarray, risks: np.ndarray, percentile: float = 1.0
) -> float:
    """Computes the concordance index for time-dependent data."""
    threshold_time = np.percentile(times, percentile * 100)
    concordant_pairs, comparable_pairs = 0, 0
    for i, ti in enumerate(times):
        for j, tj in enumerate(times):
            if events[i] == 1 and ti < tj and ti < threshold_time:
                comparable_pairs += 1
                if risks[i] > risks[j]:
                    concordant_pairs += 1
    return concordant_pairs / comparable_pairs if comparable_pairs > 0 else 0.0

In [5]:
class SurvModel(ABC):
    """Base class for survival models."""

    @abstractmethod
    def fit(self, X_train, y_train):
        """Fits the model to the training data.

        Args:
            X_train: Training data of shape (n_samples, n_features).
            y_train: Training labels of shape (n_samples,) with dtype=[("event", bool), ("time", float)].
        """
        pass

    @abstractmethod
    def predict(self, X_test, times) -> np.array:
        """Predicts the survival function for the given times.

        Args:
            X_test: Test data of shape (n_samples, n_features).
            times: Times at which to predict the survival function of shape (n_times,).

        Returns:
            Survival function of shape (n_samples, n_times).
        """
        pass

    def evaluate(self, X_test, y_test, y_train) -> dict[str, float]:
        """Evaluates the model on the test data.

        Args:
            X_test: Test data of shape (n_samples, n_features).
            y_test: Test labels of shape (n_samples,) with dtype=[("event", bool), ("time", float)].
            y_train: Training labels of shape (n_samples,) with dtype=[("event", bool), ("time", float)].

        Returns:
            Dictionary of survival metrics.
        """
        min_time, max_time = y_test["time"].min(), y_test["time"].max()
        tolerance = 0.1 * (max_time - min_time)
        times = np.linspace(min_time + tolerance, max_time - tolerance, 100)
        survs = self.predict(X_test, times)
        mean_times = survs.sum(axis=1)
        c25 = concordance_index_td(y_test["event"], y_test["time"], -mean_times, 0.25)
        c50 = concordance_index_td(y_test["event"], y_test["time"], -mean_times, 0.50)
        c75 = concordance_index_td(y_test["event"], y_test["time"], -mean_times, 0.75)
        cid = concordance_index_td(y_test["event"], y_test["time"], -mean_times)
        try:
            ibs = integrated_brier_score(y_train, y_test, survs, times)
        except ValueError as e:
            ibs = 0.25
        return {
            "cid": cid,
            "ibs": ibs,
            "c25": c25,
            "c50": c50,
            "c75": c75,
        }

In [6]:
class Cox(SurvModel):
    """Cox proportional hazards model (Cox, 1972)."""

    def __init__(self) -> None:
        self.model = CoxPHSurvivalAnalysis(alpha=0.01)
        self.failed_opt = False

    def fit(self, X_train, y_train):
        try:
            self.model.fit(X_train, y_train)
        except ValueError as e:
            self.failed_opt = True

    def predict(self, X_test, times) -> np.array:
        if self.failed_opt:
            return np.ones((X_test.shape[0], len(times))) * 0.5
        return np.array([S(times) for S in self.model.predict_survival_function(X_test)])

In [7]:
class RandomSurvivalForest(SurvModel):
    """Random survival forest model (Ishwaran et al., 2008)."""

    def __init__(self) -> None:
        self.model = RSF(n_jobs=-1)

    def fit(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, X_test, times) -> np.array:
        return np.array([S(times) for S in self.model.predict_survival_function(X_test)])

In [8]:
class DeepSurv(SurvModel):
    """DeepSurv model (Katzman et al., 2018)."""

    def __init__(self) -> None:
        self.net = None
        self.model = None

    def fit(self, X_train, y_train):
        net = tt.practical.MLPVanilla(
            X_train.shape[1],
            [i * X_train.shape[1] for i in [3, 5, 3]],  # hidden layers as in Katzman et al., 2018
            1,  # outputs
            False,  # batch norm
            0.6,  # dropout, as in Katzman et al., 2018
        )
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.15, stratify=y_train["event"]
        )
        _y_val = (y_val["time"].copy(), y_val["event"].copy())
        val_data = (X_val, _y_val)
        callbacks = [tt.callbacks.EarlyStopping()]
        self.model = CoxPH(net, tt.optim.Adam)
        _y_train = (y_train["time"].copy(), y_train["event"].copy())
        self.model.fit(
            X_train, _y_train, 256, 256, val_data=val_data, callbacks=callbacks, verbose=False
        )
        self.model.compute_baseline_hazards()

    def predict(self, X_test, times) -> np.array:
        preds = self.model.predict_surv_df(X_test)
        unique_times = preds.index.to_numpy()
        survs = np.array([preds.iloc[:, i].values for i in range(preds.shape[1])])
        surv = np.zeros((X_test.shape[0], len(times)))
        for i, t in enumerate(times):
            idx = np.abs(unique_times - t).argmin()
            surv[:, i] = survs[:, idx]
        return surv

In [9]:
class DeepHit(SurvModel):
    """DeepHit model (Lee et al., 2018)."""

    def __init__(self, num_durations: int = 5) -> None:
        self.net = None
        self.model = None
        self.labtrans = None
        self.num_durations = num_durations

    def fit(self, X_train, y_train):
        X_train, X_val, y_train, y_val = train_test_split(
            X_train, y_train, test_size=0.15, stratify=y_train["event"]
        )
        self.labtrans = DeepHitSingle.label_transform(self.num_durations)
        _y_train = self.labtrans.fit_transform(y_train["time"], y_train["event"])
        _y_val = self.labtrans.transform(y_val["time"], y_val["event"])
        val_data = (X_val, _y_val)
        net = tt.practical.MLPVanilla(
            X_train.shape[1],
            [i * X_train.shape[1] for i in [3, 5, 3]],
            self.num_durations,  # outputs
            False,  # batch norm
            0.6,  # dropout
        )
        callbacks = [tt.callbacks.EarlyStopping()]
        self.model = DeepHitSingle(net, tt.optim.Adam, alpha=0.5, duration_index=self.labtrans.cuts)
        self.model.fit(
            X_train, _y_train, 256, 256, val_data=val_data, callbacks=callbacks, verbose=False
        )

    def predict(self, X_test, times) -> np.array:
        preds = self.model.predict_surv_df(X_test)
        unique_times = preds.index.to_numpy()
        survs = np.array([preds.iloc[:, i].values for i in range(preds.shape[1])])
        surv = np.zeros((X_test.shape[0], len(times)))
        for i, t in enumerate(times):
            idx = np.abs(unique_times - t).argmin()
            surv[:, i] = survs[:, idx]
        return surv

In [10]:
class DeepSurvivalMachines(SurvModel):
    """Deep survival machines model (Nagpal et al., 2021)."""

    def __init__(self) -> None:
        self.model = None

    def fit(self, X_train, y_train):
        self.model = DSM(layers=[i * X_train.shape[1] for i in [3, 5, 3]])
        self.model.fit(X_train, y_train["time"], y_train["event"])

    def predict(self, X_test, times) -> np.array:
        r = np.concatenate(
            [
                self.model.predict_risk(X_test.astype(np.float64), t.astype(np.float64))
                for t in times
            ],
            axis=1,
        )
        survs = np.exp(-r)
        return survs

In [11]:
class CoxBoost(SurvModel):
    """CoxBoost model (Ridgeway, 1999)."""

    def __init__(self) -> None:
        self.model = GradientBoostingSurvivalAnalysis()

    def fit(self, X_train, y_train):
        self.model.fit(X_train, y_train)

    def predict(self, X_test, times) -> np.array:
        return np.array([S(times) for S in self.model.predict_survival_function(X_test)])

In [12]:
class FPBoost(SurvModel):
    """FPBoost model for AAAI submission.

    Args:
        weibull_heads: Number of Weibull heads.
        loglogistic_heads: Number of log-logistic heads.
        n_estimators: Number of base learners per estimated parameter.
        max_depth: Maximum depth of the base learners.
        learning_rate: Learning rate for the boosting algorithm.
        alpha: ElasticNet regularization strength.
        l1_ratio: Ratio between L1 and L2 regularization.
        uniform_heads: Whether to use uniform weights for the heads.
        heads_activation: Activation function for the heads. Can be "relu" or "softmax".
        patience: Patience for early stopping.
        verbose: Whether to print progress.
    """

    def __init__(
        self,
        weibull_heads: int,
        loglogistic_heads: int,
        n_estimators: int,
        max_depth: int,
        learning_rate: float,
        alpha: float,
        l1_ratio: float,
        uniform_heads: bool,
        heads_activation: str,
        patience: Optional[int],
        verbose: bool = False,
    ):
        self.weibull_heads = weibull_heads
        self.loglogistic_heads = loglogistic_heads
        self.heads = weibull_heads + loglogistic_heads
        if self.heads == 0:
            self.weibull_heads = 1
            self.heads = 1
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.learning_rate = learning_rate
        self.alpha = alpha
        self.l1_ratio = l1_ratio
        self.uniform_heads = uniform_heads
        self.heads_activation = heads_activation
        self.patience = patience
        self.verbose = verbose

        # Random initialization of the parameters
        self.init_eta = np.random.rand(self.heads) + 0.5
        self.eta_heads = [[] for _ in range(self.heads)]
        self.init_k = np.random.rand(self.heads) * 2
        self.k_heads = [[] for _ in range(self.heads)]
        self.init_w = np.random.rand(self.heads)
        self.w_heads = [[] for _ in range(self.heads)]

        heads_activation_fns = {
            "relu": lambda w: F.relu(w),
            "softmax": lambda w: F.softmax(w, dim=1),
        }
        if heads_activation not in heads_activation_fns:
            raise ValueError(f"Heads activation function not in {heads_activation_fns.keys()}")
        self.heads_activation_fn = heads_activation_fns[heads_activation]

    def _predict_etas(self, X: np.array) -> np.array:
        output = np.zeros((len(X), self.heads)) + self.init_eta.reshape((1, -1))
        for i, regs in enumerate(self.eta_heads):
            if len(regs) == 0:
                continue
            preds = np.concatenate([reg.predict(X).reshape((-1, 1)) for reg in regs], axis=1)
            output[:, i] += self.learning_rate * np.sum(preds, axis=1)
        return output

    def _predict_ks(self, X: np.array) -> np.array:
        output = np.ones((len(X), self.heads)) * self.init_k.reshape((1, -1))
        for i, regs in enumerate(self.k_heads):
            if len(regs) == 0:
                continue
            preds = np.concatenate([reg.predict(X).reshape((-1, 1)) for reg in regs], axis=1)
            output[:, i] += self.learning_rate * np.sum(preds, axis=1)
        return output

    def _predict_ws(self, X: np.array) -> np.array:
        if self.uniform_heads:
            return np.ones((len(X), self.heads)) / self.heads
        output = np.ones((len(X), self.heads)) * self.init_w.reshape((1, -1))
        for i, regs in enumerate(self.w_heads):
            if len(regs) == 0:
                continue
            preds = np.concatenate([reg.predict(X).reshape((-1, 1)) for reg in regs], axis=1)
            output[:, i] += self.learning_rate * np.sum(preds, axis=1)
        return output

    def _predict_params(self, X: np.array) -> np.array:
        etas = self._predict_etas(X).reshape((-1, self.heads, 1))
        ks = self._predict_ks(X).reshape((-1, self.heads, 1))
        ws = self._predict_ws(X).reshape((-1, self.heads, 1))
        return np.concatenate([etas, ks, ws], -1)

    def _weibull_hazard(self, eta, k, times):
        return k * eta * times ** (k - 1)

    def _weibull_cum_hazard(self, eta, k, times):
        return eta * times**k

    def _loglogistic_hazard(self, eta, k, times):
        return eta * k * times ** (k - 1) / (1 + eta * times**k)

    def _loglogistic_cum_hazard(self, eta, k, times):
        if torch.is_tensor(times):
            return torch.log1p(eta * times**k)
        return np.log1p(eta * times**k)

    def _get_neg_grads(self, params: np.array, events: Tensor, times: Tensor) -> np.array:
        params_torch = Variable(torch.tensor(params).float(), requires_grad=True)

        etas = F.relu(params_torch[:, :, 0])
        ks = F.relu(params_torch[:, :, 1])
        ws = self.heads_activation_fn(params_torch[:, :, 2])

        hazard = torch.zeros(len(times))
        cum_hazard = torch.zeros(len(times))

        if self.weibull_heads > 0:
            weibull_hazard = self._weibull_hazard(
                etas[:, : self.weibull_heads], ks[:, : self.weibull_heads], times
            )
            weibull_cum_hazard = self._weibull_cum_hazard(
                etas[:, : self.weibull_heads], ks[:, : self.weibull_heads], times
            )
            hazard += (weibull_hazard * ws[:, : self.weibull_heads]).sum(dim=1)
            cum_hazard += (weibull_cum_hazard * ws[:, : self.weibull_heads]).sum(dim=1)

        if self.loglogistic_heads > 0:
            loglogistic_hazard = self._loglogistic_hazard(
                etas[:, self.weibull_heads :], ks[:, self.weibull_heads :], times
            )
            loglogistic_cum_hazard = self._loglogistic_cum_hazard(
                etas[:, self.weibull_heads :], ks[:, self.weibull_heads :], times
            )
            hazard += (loglogistic_hazard * ws[:, self.weibull_heads :]).sum(dim=1)
            cum_hazard += (loglogistic_cum_hazard * ws[:, self.weibull_heads :]).sum(dim=1)

        log_likelihood = (events * torch.log(hazard) - cum_hazard).mean()
        l1_reg = torch.abs(params_torch).mean()
        l2_reg = (params_torch**2).mean()
        elastic_net_reg = self.l1_ratio * l1_reg + (1 - self.l1_ratio) * l2_reg
        loss = -log_likelihood + self.alpha * elastic_net_reg

        loss.backward()
        grad = params_torch.grad.numpy()
        grad[np.isnan(grad)] = 0.0
        return -(grad / np.abs(grad).max())

    def _fit_base_learner(self, X: np.array, y: np.array):
        reg = DecisionTreeRegressor(max_depth=self.max_depth)
        reg.fit(X, y)
        return reg

    def fit(self, X_train: np.array, y_train: np.array) -> None:
        if self.verbose:
            print(f"Fitting a Survival Boosting model with {self.heads} heads...")

        patience_counter, best_num_base_learners, best_cid = 0, 0, 0.0
        if self.patience is not None:
            X_train, X_val, y_train, y_val = train_test_split(
                X_train,
                y_train,
                test_size=0.2,
                stratify=y_train["event"],
            )

        events = torch.tensor(y_train["event"].copy()).float().reshape((-1,))
        times = torch.tensor(y_train["time"].copy()).float().reshape((-1, 1))
        timeline = np.linspace(np.min(y_train["time"]), np.max(y_train["time"]), 100)

        for j in range(self.n_estimators):
            params = self._predict_params(X_train)

            neg_grads = self._get_neg_grads(params, events, times)
            eta_grads = neg_grads[:, :, 0]
            k_grads = neg_grads[:, :, 1]
            w_grads = neg_grads[:, :, 2]

            for i in range(self.heads):
                self.eta_heads[i].append(self._fit_base_learner(X_train, eta_grads[:, i]))
                self.k_heads[i].append(self._fit_base_learner(X_train, k_grads[:, i]))
                if not self.uniform_heads:
                    self.w_heads[i].append(self._fit_base_learner(X_train, w_grads[:, i]))

            if self.patience is not None:
                survs = self.predict(X_val, timeline)
                mean_times = survs.sum(axis=1)
                cid = concordance_index_td(y_val["event"], y_val["time"], -mean_times)
                if self.verbose:
                    print(f"[Iteration {j:04}] Concordance: {cid:.4f}")
                if cid > best_cid:
                    best_cid = cid
                    best_num_base_learners = len(self.eta_heads[0])
                    patience_counter = 0
                else:
                    patience_counter += 1
                    if patience_counter >= self.patience:
                        break

        if self.patience is not None:
            self.eta_heads = [heads[:best_num_base_learners] for heads in self.eta_heads]
            self.k_heads = [heads[:best_num_base_learners] for heads in self.k_heads]
            self.w_heads = [heads[:best_num_base_learners] for heads in self.w_heads]

    def predict(self, X_test, times) -> np.array:
        times = times.reshape((1, 1, -1))
        params = torch.tensor(self._predict_params(X_test)).float()

        etas = F.relu(params[:, :, 0]).numpy().reshape((-1, self.heads, 1))
        ks = F.relu(params[:, :, 1]).numpy().reshape((-1, self.heads, 1))
        ws = self.heads_activation_fn(params[:, :, 2]).numpy().reshape((-1, self.heads, 1))

        cum_hazard = np.zeros((len(X_test), len(times[0][0])))

        if self.weibull_heads > 0:
            weibull_cum_hazard = self._weibull_cum_hazard(
                etas[:, : self.weibull_heads], ks[:, : self.weibull_heads], times
            )
            cum_hazard += (weibull_cum_hazard * ws[:, : self.weibull_heads]).sum(axis=1)

        if self.loglogistic_heads > 0:
            loglogistic_cum_hazard = self._loglogistic_cum_hazard(
                etas[:, self.weibull_heads :], ks[:, self.weibull_heads :], times
            )
            cum_hazard += (loglogistic_cum_hazard * ws[:, self.weibull_heads :]).sum(axis=1)
        surv = np.exp(-cum_hazard)
        return surv

In [13]:
def init_model(model: str, params: dict) -> SurvModel:
    """Initializes a survival model with the given parameters.

    Args:
        model: String with the model name.
        params: Dictionary with the model parameters.

    Returns:
        Initialized survival model.
    """

    models = {
        "cox": Cox,
        "rsf": RandomSurvivalForest,
        "deepsurv": DeepSurv,
        "deephit": DeepHit,
        "dsm": DeepSurvivalMachines,
        "coxboost": CoxBoost,
        "fpboost": FPBoost,
    }
    return models[model](**params)

## Training

In [14]:
def atomic_training(dataset: str, model: str, params: dict) -> dict[str, float]:
    """Trains and evaluates a model on a dataset."""

    np.random.seed(SEED)
    torch.manual_seed(SEED)

    df = load_dataframe(dataset)

    metrics = {}
    for train, val, test in get_k_fold_splits(df):
        X_train, y_train = train
        X_val, y_val = val
        X_test, y_test = test

        m = init_model(model, params)
        m.fit(X_train, y_train)
        val_results = m.evaluate(X_val, y_val, y_train)
        test_results = m.evaluate(X_test, y_test, y_train)

        for k, v in val_results.items():
            k = f"{k}_val"
            if k not in metrics:
                metrics[k] = []
            metrics[k].append(v)
        for k, v in test_results.items():
            k = f"{k}_test"
            if k not in metrics:
                metrics[k] = []
            metrics[k].append(v)

    ret = {}
    for k, v in metrics.items():
        ret[f"{k}_mean"] = np.mean(v).item()
        ret[f"{k}_std"] = np.std(v).item()

    return ret

## Results

Train and evaluate the baseline models and FPBoost on the datasets.

In [15]:
NUM_CPUS = 8  # Number of CPUs for parallel training
OBJ_MEMORY_GB = 2  # Memory for each ray object in GB

RESULTS_PATH = "results"  # Path to save the results

# Datasets on which to train the models
DATASETS = [
    "aids",
    "breast_cancer",
    "gbsg",
    "metabric",
    "support",
    "veterans",
    "whas",
]

# Baseline models to train
BASELINE_MODELS = ["rsf", "cox", "coxboost", "deepsurv", "dsm", "deephit"]

# Search space for the hyperparameter optimization of the FPBoost model
SEARCH_SPACE = {
    "weibull_heads": tune.randint(0, 8),
    "loglogistic_heads": tune.randint(0, 8),
    "n_estimators": tune.randint(1, 256),
    "max_depth": tune.randint(1, 8),
    "learning_rate": tune.uniform(1e-2, 1),
    "alpha": tune.uniform(0.0, 1.0),
    "l1_ratio": tune.uniform(0, 1),
    "uniform_heads": tune.choice([True, False]),
    "heads_activation": tune.choice(["relu", "softmax"]),
    "patience": tune.choice([None, 4, 16]),
}

ITERATIONS = 8  # Number of iterations for the hyperparameter optimization

In [16]:
os.makedirs(RESULTS_PATH, exist_ok=True)
ray.init(
    num_cpus=NUM_CPUS,
    object_store_memory=OBJ_MEMORY_GB * 1024 * 1024 * 1024,
    ignore_reinit_error=True,
)

2024-08-18 11:12:33,950	INFO worker.py:1740 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 


Python version:,3.10.13
Ray version:,2.20.0
Dashboard:,http://127.0.0.1:8265


### Baselines

In [17]:
tempdir = os.path.join(RESULTS_PATH, "temp_baselines")
os.makedirs(tempdir, exist_ok=True)


@ray.remote
def remote_baseline_training(dataset: str, model: str) -> dict[str, float]:
    if os.path.exists(os.path.join(tempdir, f"{dataset}_{model}.json")):
        return None
    ret = atomic_training(dataset, model, {})
    ret["dataset"] = dataset
    ret["model"] = model
    ret["params"] = {}
    with open(os.path.join(tempdir, f"{dataset}_{model}.json"), "w") as f:
        json.dump(ret, f)
    return ret


BASELINE_RESULTS_FILE = os.path.join(RESULTS_PATH, "baseline_results.csv")

baseline_results = []
for dataset in DATASETS:
    for model in BASELINE_MODELS:
        baseline_results.append(remote_baseline_training.remote(dataset, model))
baseline_results = ray.get(baseline_results)

results = {}
for f in os.listdir(tempdir):
    with open(os.path.join(tempdir, f), "r") as file:
        r = json.load(file)
        for k, v in r.items():
            if k not in results:
                results[k] = []
            results[k].append(v)

baseline_results_df = pd.DataFrame(results)
baseline_results_df.to_csv(BASELINE_RESULTS_FILE, index=False)

  1%|▏         | 138/10000 [00:00<00:17, 552.85it/s]
(remote_baseline_training pid=51961) /home/aarchetto/.miniconda3/envs/fpboost/lib/python3.10/site-packages/torchtuples/base.py:669: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitH

### FPBoost

In [18]:
def objective(config):
    return atomic_training(config["dataset"], "fpboost", config["params"])


os.makedirs(RESULTS_PATH, exist_ok=True)

for dataset in DATASETS:
    print(f"Training FPBoost on {dataset}...")

    search_alg = OptunaSearch(
        metric="cid_val_mean", mode="max", sampler=optuna.samplers.TPESampler()
    )

    analysis = tune.run(
        objective,
        config={"dataset": dataset, "params": SEARCH_SPACE},
        num_samples=ITERATIONS,
        search_alg=search_alg,
        name=f"{dataset}_{model}_tune_optuna_experiment",
        storage_path=f"file://{os.path.abspath(RESULTS_PATH)}",
    )

    df = analysis.results_df
    csv_filename = f"{dataset}_{model}_tune_optuna_experiment.csv"
    df.to_csv(os.path.join(RESULTS_PATH, csv_filename), index=False)

2024-08-18 11:18:48,227	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949
[I 2024-08-18 11:18:48,249] A new study created in memory with name: optuna
2024-08-18 11:18:48,250	INFO tensorboardx.py:193 -- pip install "ray[tune]" to see TensorBoard files.
2024-08-18 11:18:48,251	WARNING callback.py:136 -- The TensorboardX logger cannot be instantiated because either TensorboardX or one of it's dependencies is not installed. Please make sure you have the latest version of TensorboardX installed: `pip install -U tensorboardx`


Training FPBoost on aids...


Trial name,c25_test_mean,c25_test_std,c25_val_mean,c25_val_std,c50_test_mean,c50_test_std,c50_val_mean,c50_val_std,c75_test_mean,c75_test_std,c75_val_mean,c75_val_std,cid_test_mean,cid_test_std,cid_val_mean,cid_val_std,ibs_test_mean,ibs_test_std,ibs_val_mean,ibs_val_std
objective_030f5ec2,0.711244,0.063081,0.722467,0.0865636,0.718439,0.0580107,0.7156,0.0774847,0.712893,0.0542642,0.707655,0.0681003,0.7137,0.0548716,0.708344,0.0689261,0.0847562,0.0165996,0.0843827,0.0184769
objective_27909715,0.710526,0.0764107,0.71414,0.0698132,0.711986,0.0719895,0.715287,0.0610828,0.7058,0.06483,0.706824,0.0528903,0.706091,0.0651657,0.706933,0.0529778,0.094289,0.031096,0.0938427,0.0294747
objective_36f397fd,0.589392,0.131793,0.603995,0.126347,0.590961,0.118198,0.599584,0.111533,0.581547,0.105931,0.593025,0.0999987,0.581141,0.105351,0.593237,0.100355,0.158908,0.0193719,0.158633,0.0154968
objective_4f1ab806,0.713805,0.0694376,0.740193,0.0603781,0.718512,0.0672486,0.731991,0.0514019,0.712714,0.0613158,0.727547,0.0470914,0.713262,0.0619958,0.72782,0.0474309,0.0833519,0.0246785,0.0827952,0.0238931
objective_79031809,0.613298,0.151641,0.5859,0.176386,0.620296,0.152842,0.582793,0.164807,0.615538,0.147199,0.588024,0.152058,0.616242,0.148008,0.587728,0.152687,0.149652,0.0259409,0.149304,0.0232495
objective_97acf6a5,0.629724,0.143979,0.604001,0.127362,0.638236,0.106833,0.617009,0.106645,0.635843,0.11136,0.609975,0.107809,0.637179,0.110837,0.611194,0.108014,0.0651886,0.00627284,0.0646789,0.00564344
objective_9b628a74,0.643196,0.0897293,0.62431,0.100402,0.645936,0.0768703,0.623606,0.0871395,0.641937,0.0810677,0.624289,0.0868508,0.641817,0.0809981,0.625232,0.0863893,0.0725347,0.00992678,0.0700942,0.00896586
objective_e65d6e14,0.64687,0.0976466,0.62745,0.117118,0.635102,0.103915,0.633728,0.108529,0.636569,0.100749,0.629152,0.10935,0.637357,0.10137,0.629214,0.109451,0.118403,0.0455868,0.11687,0.0355414


2024-08-18 11:21:02,084	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-08-18 11:21:02,086	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/home/aarchetto/fpboost/results/aids_deephit_tune_optuna_experiment' in 0.0044s.
2024-08-18 11:21:02,090	INFO tune.py:1039 -- Total run time: 133.86 seconds (133.82 seconds for the tuning loop).
2024-08-18 11:21:02,098	INFO tune.py:614 -- [output] This uses the legacy output and progress reporter

Training FPBoost on breast_cancer...


...

Trial name,c25_test_mean,c25_test_std,c25_val_mean,c25_val_std,c50_test_mean,c50_test_std,c50_val_mean,c50_val_std,c75_test_mean,c75_test_std,c75_val_mean,c75_val_std,cid_test_mean,cid_test_std,cid_val_mean,cid_val_std,ibs_test_mean,ibs_test_std,ibs_val_mean,ibs_val_std
objective_14ec9601,0.746388,0.118224,0.795202,0.116133,0.725189,0.110574,0.738041,0.108233,0.706321,0.0860732,0.71079,0.0890071,0.695464,0.084887,0.702496,0.0803405,0.39488,0.0733317,0.391701,0.0406926
objective_15fe14d0,0.708406,0.137772,0.774875,0.178418,0.6889,0.111285,0.720502,0.131193,0.69114,0.0632125,0.705149,0.101313,0.685193,0.0587538,0.698163,0.0972734,0.399377,0.0572676,0.403233,0.0515495
objective_4bafa406,0.720239,0.130665,0.812771,0.12263,0.700235,0.109995,0.746744,0.0980281,0.684101,0.0872474,0.730584,0.0925268,0.676782,0.0832881,0.723047,0.0852726,0.429905,0.0559241,0.430748,0.05481
objective_7f60b581,0.744225,0.0825206,0.739278,0.123468,0.703175,0.0957607,0.687586,0.0864743,0.693376,0.0796169,0.688251,0.065684,0.674889,0.0784422,0.673486,0.0624012,0.440311,0.0711464,0.444317,0.0638064
objective_e33d38ac,0.635402,0.140536,0.748004,0.162188,0.624439,0.124182,0.714421,0.137183,0.6334,0.0892019,0.708153,0.112854,0.629659,0.0849074,0.700454,0.10725,0.159197,0.0357619,0.14774,0.044903
objective_ee189bcc,0.787361,0.0992758,0.768542,0.0703193,0.753172,0.112549,0.725959,0.0756275,0.724978,0.0793919,0.71345,0.0547462,0.716991,0.08023,0.703226,0.0494312,0.412951,0.0707479,0.420469,0.0660487
objective_fafda5be,0.741785,0.181577,0.754513,0.190382,0.71143,0.185929,0.70637,0.139632,0.69412,0.136698,0.694413,0.109615,0.686704,0.129683,0.680995,0.101268,0.31681,0.0539642,0.320644,0.04248
objective_febe0179,0.722692,0.141838,0.742361,0.130633,0.677414,0.125394,0.698242,0.123985,0.656226,0.101543,0.680334,0.103767,0.644514,0.100669,0.668091,0.100265,0.421278,0.0644195,0.426871,0.0621192


2024-08-18 11:42:22,095	WARNING experiment_state.py:205 -- Experiment state snapshotting has been triggered multiple times in the last 5.0 seconds. A snapshot is forced if `CheckpointConfig(num_to_keep)` is set, and a trial has checkpointed >= `num_to_keep` times since the last snapshot.
You may want to consider increasing the `CheckpointConfig(num_to_keep)` or decreasing the frequency of saving checkpoints.
You can suppress this error by setting the environment variable TUNE_WARN_EXCESSIVE_EXPERIMENT_CHECKPOINT_SYNC_THRESHOLD_S to a smaller value than the current threshold (5.0).
2024-08-18 11:42:22,097	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/home/aarchetto/fpboost/results/veterans_deephit_tune_optuna_experiment' in 0.0044s.
2024-08-18 11:42:22,101	INFO tune.py:1039 -- Total run time: 131.44 seconds (131.41 seconds for the tuning loop).
2024-08-18 11:42:22,110	INFO tune.py:614 -- [output] This uses the legacy output and progress repo

Training FPBoost on whas...


Trial name,c25_test_mean,c25_test_std,c25_val_mean,c25_val_std,c50_test_mean,c50_test_std,c50_val_mean,c50_val_std,c75_test_mean,c75_test_std,c75_val_mean,c75_val_std,cid_test_mean,cid_test_std,cid_val_mean,cid_val_std,ibs_test_mean,ibs_test_std,ibs_val_mean,ibs_val_std
objective_3828efa5,0.73013,0.0442217,0.738555,0.0542862,0.736346,0.0314687,0.744332,0.0450872,0.734004,0.0371632,0.743419,0.0511094,0.731018,0.0401101,0.740928,0.0519408,0.202609,0.0143982,0.200477,0.0111068
objective_4f7d97e5,0.722613,0.0672724,0.703388,0.0413983,0.733538,0.0554441,0.712102,0.0474338,0.732282,0.056179,0.714839,0.0527751,0.726611,0.058049,0.713407,0.0539287,0.201157,0.024388,0.20281,0.0164189
objective_69fdd71b,0.738535,0.0625118,0.742742,0.0593157,0.751623,0.0541111,0.752664,0.0565154,0.749046,0.0602356,0.751521,0.0613117,0.745658,0.0642253,0.750053,0.0622439,0.169476,0.0202114,0.167675,0.0182922
objective_6efb437d,0.715046,0.0308518,0.707489,0.0620437,0.722469,0.0385187,0.715308,0.0553727,0.715426,0.0418098,0.715467,0.0571659,0.712777,0.0436146,0.713093,0.0564387,0.203247,0.0143096,0.202483,0.0178346
objective_90010fbb,0.748932,0.0661283,0.73279,0.0705882,0.757758,0.0536357,0.743474,0.0586248,0.757565,0.0553417,0.746338,0.0601548,0.755293,0.0549029,0.745418,0.0583954,0.171945,0.0169052,0.174763,0.0192942
objective_cac9b08e,0.710925,0.0740448,0.724653,0.0667755,0.72209,0.0601907,0.734161,0.0594376,0.718764,0.0693276,0.734843,0.063161,0.718396,0.0660389,0.732963,0.065417,0.202178,0.0126567,0.201646,0.0131505
objective_ccadceff,0.733755,0.0632353,0.73466,0.0508425,0.742663,0.0528474,0.752559,0.0497361,0.740004,0.0609799,0.754195,0.052528,0.737683,0.0586882,0.752923,0.0523992,0.173077,0.0298953,0.166172,0.0274646
objective_f6ed7fef,0.735445,0.0544197,0.712427,0.0661281,0.737037,0.0446968,0.723318,0.054378,0.731417,0.0487122,0.718403,0.0610318,0.729244,0.051358,0.71847,0.0624159,0.208924,0.0325475,0.213011,0.0547398


2024-08-18 11:44:07,944	INFO tune.py:1007 -- Wrote the latest version of all result files and experiment state to '/home/aarchetto/fpboost/results/whas_deephit_tune_optuna_experiment' in 0.0035s.
2024-08-18 11:44:07,947	INFO tune.py:1039 -- Total run time: 105.84 seconds (105.81 seconds for the tuning loop).


In [19]:
fpboost_results = []
for file in os.listdir(RESULTS_PATH):
    if file.endswith("_tune_optuna_experiment.csv"):
        df = pd.read_csv(os.path.join(RESULTS_PATH, file))
        fpboost_results.append(df)

df = pd.concat(fpboost_results)
df["model"] = "fpboost"
df = df.dropna(axis=1, how="any")
df.columns = df.columns.str.replace("config/", "")
df.reset_index(drop=True, inplace=True)

# Select the best hyperparameters for each dataset accrding to the C-Index - IBS difference on the validation set
df["sel_col"] = df["cid_val_mean"] - df["ibs_val_mean"]
idx = df.groupby(["dataset", "model"])["sel_col"].idxmax()
df = df.loc[idx]
assert df.groupby(["dataset", "model"]).size().eq(1).all()

df = df.drop(
    columns=[
        "sel_col",
        "timestamp",
        "time_since_restore",
        "pid",
        "time_total_s",
        "date",
        "training_iteration",
        "time_this_iter_s",
        "done",
        "hostname",
        "node_ip",
        "iterations_since_restore",
        "experiment_tag",
    ]
)

FPBOOST_RESULTS_FILE = os.path.join(RESULTS_PATH, "fpboost_results.csv")

df.to_csv(FPBOOST_RESULTS_FILE, index=False)

### Results Collection

Load the results returned by the previous cells.

In [20]:
baseline_results_df = pd.read_csv(BASELINE_RESULTS_FILE)
fpboost_results_df = pd.read_csv(FPBOOST_RESULTS_FILE)

results_df = pd.concat([baseline_results_df, fpboost_results_df])
results_df.sort_values(["dataset", "model"], inplace=True)
results_df.to_csv(os.path.join(RESULTS_PATH, "results.csv"), index=False)